In [28]:
import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from matplotlib.ticker import PercentFormatter

# Step 1: Load the CSV file
df = pd.read_csv('../data/combined_stocks_splits.csv')

# Step 2: Extract stock symbols dynamically from column names
stock_symbols = ['amzn', 'goog', 'meta', 'nvda', 'orcl', 'tsla']

# Step 3: Prepare a DataFrame to store percentage changes and day of the week
daily_pct_changes = pd.DataFrame()
daily_pct_changes['date'] = pd.to_datetime(df['date'])  # Add the date column
daily_pct_changes['day_of_week'] = daily_pct_changes['date'].dt.dayofweek  # Day of week (0=Mon, 4=Fri)

# Step 4: Calculate daily percentage changes for each stock
for symbol in stock_symbols:
    close_col = f'close ({symbol})'  # Identify the close column for the current stock
    daily_pct_changes[symbol] = df[close_col].pct_change()  # Daily percentage change

# Step 5: Group by day of the week and calculate mean for all stocks
grouped_data = daily_pct_changes.groupby('day_of_week')[stock_symbols].mean().reset_index()

# Step 6: Melt the data for easier plotting
melted_data = grouped_data.melt(id_vars='day_of_week', 
                                value_vars=stock_symbols, 
                                var_name='Stock', 
                                value_name='Average % Change')

# Step 7: Build the Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Weekly Average Daily Percentage Change by Stock", style={'textAlign': 'center'}),
    
    # Dropdown filter with "Select All" option
    html.Label("Select Stocks:"),
    dcc.Dropdown(
        id='stock-filter',
        options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': stock.upper(), 'value': stock} for stock in stock_symbols],
        value='ALL',  # Default to all stocks
        multi=False,  # Single or "ALL" selection
    ),
    
    # Bar chart output
    dcc.Graph(id='bar-chart')
])

@app.callback(
    Output('bar-chart', 'figure'),
    Input('stock-filter', 'value')
)
def update_chart(selected_stock):
    # Copy the data to avoid modifications
    filtered_data = melted_data.copy()
    
    # Handle filtering logic
    if selected_stock == 'ALL':
        plot_data = filtered_data
    else:
        plot_data = filtered_data[filtered_data['Stock'] == selected_stock]
    
    # Map day_of_week integers to day names
    day_names = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri'}
    plot_data['day_of_week'] = plot_data['day_of_week'].map(day_names)
    
    # Create the bar chart
    fig = px.bar(
        plot_data,
        x='day_of_week',
        y='Average % Change',
        color='Stock',
        title="Average Daily Percentage Change by Stock",
        labels={'day_of_week': 'Day of the Week', 'Average % Change': 'Average % Change (%)'},
        barmode='group',
        height=600
    )
    
    fig.update_layout(
    xaxis_title="Day of the Week",
    yaxis=dict(title="Average % Change"),
    template="plotly"
    )

    fig.update_layout(yaxis_tickformat = ".2%")


    return fig

# Step 8: Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\brown\AppData\Local\Temp\ipykernel_10972\2635311386.py:67: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brown\AppData\Local\Temp\ipykernel_10972\2635311386.py:67: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brown\AppData\Local\Temp\ipykernel_10972\2635311386.py:67: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa